# youtube (견인사이) 크롤링 코드

In [3]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time

In [5]:
# 1. 크롬 옵션 (창 없이 실행 가능)
options = Options()
options.add_argument('--headless')  # 창 안 띄우고 실행하려면 주석 해제
options.add_argument('--disable-gpu')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

# 2. 드라이버 경로
# driver_path = 'chromedriver.exe'
# service = Service(executable_path=driver_path)
# driver = webdriver.Chrome(service=service, options=options)
driver = webdriver.Chrome()
# 3. 유튜브 채널 URL 접속
channel_url = "https://www.youtube.com/@%EA%B2%AC%EC%9D%B8%EC%82%AC%EC%9D%B4/videos"
driver.get(channel_url)
time.sleep(3)

# 4. 스크롤 내려서 더 많은 영상 로드 (선택)
SCROLL_PAUSE_TIME = 2
last_height = driver.execute_script("return document.documentElement.scrollHeight")

for _ in range(5):  # 5번 정도 스크롤 내려서 영상 추가 로드
    driver.execute_script("window.scrollTo(0, document.documentElement.scrollHeight);")
    time.sleep(SCROLL_PAUSE_TIME)
    new_height = driver.execute_script("return document.documentElement.scrollHeight")
    if new_height == last_height:
        break
    last_height = new_height

# 5. a 태그에서 video-title-link 수집
video_elements = driver.find_elements(By.ID, "video-title-link")
print(f"✅ 영상 수: {len(video_elements)}")

video_urls = []

for elem in video_elements:
    href = elem.get_attribute("href")
    title = elem.text.strip()
    if href:
        video_urls.append(href)
        print(f"🎬 {title}")
        print(f"🔗 {href}")

# 6. 저장
with open("all_video_urls.txt", "w", encoding="utf-8") as f:
    for url in video_urls:
        f.write(url + "\n")

print("\n✅ 수집 완료! 'all_video_urls.txt'에 저장됨.")

driver.quit()



✅ 영상 수: 17
🎬 강아지가 꼬리를 쫙 펴고 흔들지 않을 때
🔗 https://www.youtube.com/watch?v=zrcWijvMmsE
🎬 우리 강아지 앉아-기다려 교육하기
🔗 https://www.youtube.com/watch?v=j6HMGujBoP8
🎬 강아지끼리 서로 핥아주는 이유는?
🔗 https://www.youtube.com/watch?v=T4t6zRa3AE4
🎬 강아지의 놀이 포즈, 플레이 바우
🔗 https://www.youtube.com/watch?v=vQ9q2_gUBjA&pp=0gcJCYUJAYcqIYzv
🎬 '이리와!' 하면 오는 매너있는 강아지로 양육해요
🔗 https://www.youtube.com/watch?v=vnk0Hw_k94Y
🎬 왜 강아지는 산책하면서 자꾸 소변을 보나요?
🔗 https://www.youtube.com/watch?v=pkTsNGrFcdM
🎬 우리 강아지가 침대에 오줌을 싸서 고민이에요
🔗 https://www.youtube.com/watch?v=9C_dlK3etBE
🎬 우리 강아지가 편안한 '엎드리기' 교육하기
🔗 https://www.youtube.com/watch?v=yfv9ewCbXNA
🎬 우리 강아지 꼬리를 긴장감 없이 좌우로 흔들어요 #기분이좋아요 편
🔗 https://www.youtube.com/watch?v=yqflupcC8mQ
🎬 우리 강아지 꼬리가 축 처져 있어요
🔗 https://www.youtube.com/watch?v=v-PhyNt1i1M
🎬 우리 강아지와 '눈 마주치기'로 소통해요
🔗 https://www.youtube.com/watch?v=cVQAWNOqWVU
🎬 우리 강아지가 눈을 깜빡이면서 나를 쳐다봐요 #나를쳐다봄편
🔗 https://www.youtube.com/watch?v=w4xkJJlod5s
🎬 우리 강아지가 다른 강아지에게 민망하게 왜 이럴까요? #마운팅 편
🔗 https://www.youtube.com/watch?v=oIvYWf2laIA
🎬 우리 강아지 왜 앉아

In [7]:
import yt_dlp
import csv
from youtube_transcript_api import YouTubeTranscriptApi, TranscriptsDisabled, NoTranscriptFound

# 링크 파일 읽기
with open('all_video_urls.txt', 'r', encoding='utf-8') as f:
    video_urls = [line.strip() for line in f if line.strip()]

# 결과 저장할 CSV 파일
with open('5_youtube_data.csv', 'w', newline='', encoding='utf-8-sig') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['title', 'content', 'URL', 'image'])  # 헤더

    for url in video_urls:
        try:
            # 영상 ID 추출
            video_id = url.split('v=')[-1].split('&')[0]

            # yt-dlp로 제목, 썸네일 가져오기
            ydl_opts = {
                'skip_download': True,
                'quiet': True,
                'forcejson': True,
            }

            with yt_dlp.YoutubeDL(ydl_opts) as ydl:
                info = ydl.extract_info(url, download=False)

            title = info.get('title', '제목 없음')
            thumbnail = info.get('thumbnail', f"https://img.youtube.com/vi/{video_id}/maxresdefault.jpg")

            # 자막 가져오기
            try:
                transcript = YouTubeTranscriptApi.get_transcript(video_id, languages=['ko', 'en'])
                captions = '\n'.join([item['text'] for item in transcript])
            except (TranscriptsDisabled, NoTranscriptFound):
                captions = '자막 없음'

            # CSV 저장
            writer.writerow([title, captions, url, thumbnail])

            print(f"✅ {title} 저장 완료!")

        except Exception as e:
            print(f"⚠️ {url} 처리 실패: {e}")

print("📄 모든 유튜브 데이터 저장 완료!")

✅ 강아지가 꼬리를 쫙 펴고 흔들지 않을 때 저장 완료!
✅ 우리 강아지 앉아-기다려 교육하기 저장 완료!
⚠️ https://www.youtube.com/watch?v=T4t6zRa3AE4 처리 실패: no element found: line 1, column 0
✅ 강아지의 놀이 포즈, 플레이 바우 저장 완료!
✅ '이리와!' 하면 오는 매너있는 강아지로 양육해요 저장 완료!
✅ 왜 강아지는 산책하면서 자꾸 소변을 보나요? 저장 완료!
✅ 우리 강아지가 침대에 오줌을 싸서 고민이에요 저장 완료!
⚠️ https://www.youtube.com/watch?v=yfv9ewCbXNA 처리 실패: no element found: line 1, column 0
✅ 우리 강아지 꼬리를 긴장감 없이 좌우로 흔들어요 #기분이좋아요 편 저장 완료!
✅ 우리 강아지 꼬리가 축 처져 있어요 저장 완료!
✅ 우리 강아지와 '눈 마주치기'로 소통해요 저장 완료!
✅ 우리 강아지가 눈을 깜빡이면서 나를 쳐다봐요 #나를쳐다봄편 저장 완료!
✅ 우리 강아지가 다른 강아지에게 민망하게 왜 이럴까요? #마운팅 편 저장 완료!
✅ 우리 강아지 왜 앉아서 한 발을 들까요? #앉아서한발들기 편 저장 완료!
⚠️ https://www.youtube.com/watch?v=awtkVAmP6WY 처리 실패: no element found: line 1, column 0
✅ 우리 강아지 야단칠 땐 하품을 해요 #하품 편 저장 완료!
✅ 우리 강아지 왜 몸을 털까요? #몸털기 편 저장 완료!
📄 모든 유튜브 데이터 저장 완료!
